# установка библиотек 

In [7]:
!pip install pandas 
!pip install -U scikit-learn

импортирую библиотеки 

In [8]:
import pandas as pd
import psycopg2 
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [9]:
conn = psycopg2.connect(
    dbname="data",#то что под databases
    user="postgres",
    password="486252522",
    host="localhost",
    port="5433"
)
cursor = conn.cursor()
cursor.execute("SELECT * FROM public.expenses")
results = cursor.fetchall()

expenses1 = pd.DataFrame(results)

cursor = conn.cursor()
cursor.execute("SELECT * FROM public.orders")
results = cursor.fetchall()

orders = pd.DataFrame(results)

cursor = conn.cursor()
cursor.execute("SELECT * FROM public.visitation")
results = cursor.fetchall()

visitation = pd.DataFrame(results)

cursor = conn.cursor()
cursor.execute("SELECT * FROM public.merged_data")
results = cursor.fetchall()

merged_data1 = pd.DataFrame(results)
conn.close()

In [12]:
data=pd.read_csv('C:\\Users\\kikot\\OneDrive\\Рабочий стол\\маркетинг\\data.csv')
data.head()#вывожу таблицу

C:\Users\kikot\AppData\Local\Temp\ipykernel_22192\4141933357.py:1: DtypeWarning: Columns (0,1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('C:\\Users\\kikot\\OneDrive\\Рабочий стол\\маркетинг\\data.csv')


,dt,Channel,costs,User Id,Event Dt,Revenue,Region,Device,Session Start,Session End
0,2022-05-01,FaceBoom,113.3,1.882464e+11,2022-05-01 23:09:52,4.99,Russia,iPhone,2022-05-01 02:36:01,2022-05-01 02:45:01
1,2022-05-02,FaceBoom,78.1,1.743614e+11,2022-05-01 12:24:04,4.99,Russia,iPhone,2022-05-01 04:46:31,2022-05-01 04:47:35
2,2022-05-03,FaceBoom,85.8,5.296101e+11,2022-05-01 11:34:04,4.99,Russia,Mac,2022-05-01 14:09:25,2022-05-01 15:32:08
3,2022-05-04,FaceBoom,136.4,3.199395e+11,2022-05-01 15:34:40,4.99,Russia,Android,2022-05-01 00:29:59,2022-05-01 00:54:25
4,2022-05-05,FaceBoom,122.1,3.660003e+11,2022-05-01 13:59:51,4.99,Russia,Mac,2022-05-01 03:33:35,2022-05-01 03:57:40


перекодирую данные

In [11]:
# Подключаем класс для предобработки данных
from sklearn import preprocessing

# Напишем функцию, которая принимает на вход DataFrame, кодирует числовыми значениями категориальные признаки
# и возвращает обновленный DataFrame и сами кодировщики.
def number_encode_features(init_df):
    result = init_df.copy() # копируем нашу исходную таблицу
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == object: # np.object -- строковый тип / если тип столбца - строка, то нужно его закодировать
            encoders[column] = preprocessing.LabelEncoder() # для колонки column создаем кодировщик
            result[column] = encoders[column].fit_transform(result[column]) # применяем кодировщик к столбцу и перезаписываем столбец
    return result, encoders

df, encoders = number_encode_features(data) # Теперь encoded data содержит закодированные кат. признаки
df.head()

,dt,Channel,costs,User Id,Event Dt,Revenue,Region,Device,Session Start,Session End
0,0,1,113.3,1.882464e+11,24,4.99,1,3,99,85
1,1,1,78.1,1.743614e+11,13,4.99,1,3,169,159
2,2,1,85.8,5.296101e+11,12,4.99,1,1,508,546
3,3,1,136.4,3.199395e+11,17,4.99,1,0,27,24
4,4,1,122.1,3.660003e+11,15,4.99,1,1,128,130


#  4.1 Предсказание целевой переменной

задаю переменной X мои перекодированные данные

In [10]:
X=df
X.head()

,dt,Channel,costs,User Id,Event Dt,Revenue,Region,Device,Session Start,Session End
0,0,1,113.3,1.882464e+11,24,4.99,1,3,99,85
1,1,1,78.1,1.743614e+11,13,4.99,1,3,169,159
2,2,1,85.8,5.296101e+11,12,4.99,1,1,508,546
3,3,1,136.4,3.199395e+11,17,4.99,1,0,27,24
4,4,1,122.1,3.660003e+11,15,4.99,1,1,128,130


присваиваю переменной y колонну Channel

In [11]:
y=df['Channel']



разделяем данные на обучающую и тестовую выборки

test_size=0.2 задает размер тестовой выборки в 20% от исходных данных.

random_state=42 задает начальное значение для генератора случайных чисел, чтобы результаты можно было воспроизвести.


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

обучаю модель 

In [13]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Оценка R2 – очень важный показатель, который используется для оценки производительности модели машинного обучения на основе регрессии.

In [14]:
r2=r2_score(model.predict(X_test), y_test)
print(f'Производительность модели: {r2}')

Производительность модели: 1.0


присваиваем переменной y_pred значения для вывода точности модели 

In [15]:
y_pred = model.predict(X_test)

Accuracy — это метрика, которая описывает общую точность предсказания модели по всем классам.


In [16]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0


**Я провел обучение модели и так же использовал R2 метрику**

R2 показала ппроизводительность модели : 1.0

Я вывел значимость каждого атрибута в моем датафрейме , самым значимым признаком оказался  Channel 

Выполнил прогнозирование для контрольной выборки с

помощью полученной модели. Определил качество модели на контрольной

выборке с помощью коэффициента детерминации (R2).


# 4.2 Разработка прикладного решения

In [2]:
!pip install flask

In [16]:
from flask import Flask, request, jsonify
from sklearn.linear_model import LinearRegression
import pandas as pd

app = Flask(__name__)

# Создаем пустую модель
model = LinearRegression()

# Функция для обучения модели на основе данных из DataFrame
def train_model_from_dataframe(df):
    # Предположим, что в датафрейме df содержатся признаки (X) и целевая переменная (y)
    X = df.drop(columns=['costs'])  # Исключаем столбец 'costs' из признаков
    y = df['costs']
    
    # Обучаем модель
    model.fit(X, y)

# Обработчик для загрузки данных из DataFrame и обучения модели
@app.route('/train', methods=['POST'])
def train_model():
    # Получаем данные из запроса
    df_data = request.json
    
    # Преобразуем данные в DataFrame
    df = pd.DataFrame(df_data)
    
    # Обучаем модель на данных из DataFrame
    train_model_from_dataframe(df)
    
    return "Модель обучена успешно!"

# Обработчик для прогнозирования на основе данных из DataFrame
@app.route('/predict', methods=['POST'])
def predict():
    # Получаем данные из запроса
    input_data = request.json
    
    # Преобразуем данные в DataFrame
    df = pd.DataFrame(input_data)
    
    # Предположим, что в DataFrame содержатся признаки, на основе которых делается прогноз
    # Например, мы можем использовать все столбцы, кроме 'costs'
    X = df.drop(columns=['costs'])
    
    # Получаем прогноз от модели
    prediction = model.predict(X)
    
    # Возвращаем результат в формате JSON
    return jsonify({'predicted_costs': prediction.tolist()})

# Обработчик для получения справки
@app.route('/help', methods=['GET'])
def help():
    # Возвращаем справку по командам
    help_text = "Справка по командам:\n1. /train - Принимает POST-запрос с данными для обучения модели\n2. /predict - Принимает POST-запрос с данными для прогнозирования и возвращает прогноз\n3. /help - Возвращает справку по доступным командам"
    return help_text

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [13]:
from flask import Flask, request, jsonify

# Предположим, что у вас есть модель для прогнозирования
# и она уже загружена
# Например, вы можете использовать модуль sklearn для этой цели
from sklearn.linear_model import LinearRegression

# Создание экземпляра Flask
app = Flask(__name__)

# Пример для хранения данных
# Здесь вы можете использовать свою реальную модель
model = LinearRegression()

# Пример предполагаемых данных
# Вам нужно будет адаптировать этот код для вашего собственного случая
# Это всего лишь пример
data = {
    "dt": "2024-03-20",
    "Channel": "Direct",
    "costs": 100,
    "User_Id": 123,
    "Event_Dt": "2024-03-20",
    "Revenue": 500,
    "Region": "Europe",
    "Device": "Desktop",
    "Session_Start": "2024-03-20 10:00:00",
    "Session_End": "2024-03-20 11:00:00"
}

# Эндпоинт для получения прогноза
@app.route('/predict', methods=['POST'])
def predict():
    # Получение данных из POST-запроса
    input_data = request.json
    
    # Пример: просто вернем данные, которые были получены
    # В реальности здесь будет ваш код для обработки входных данных и генерации прогноза
    output_data = model.predict(input_data)
    
    return jsonify({"prediction": output_data})

# Эндпоинт для получения справки
@app.route('/help', methods=['GET'])
def help():
    # Пример справки по командам
    help_text = """
    Доступные команды:
    1. POST /predict - Получить прогноз, отправив данные в формате JSON
    2. GET /help - Получить эту справку
    """
    return help_text

if __name__ == '__main__':
    # Запуск сервера Flask
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\kikot\OneDrive\Рабочий стол\маркетинг\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Отчет 

**В этой сессии я прошел обучение выбранное модели**

 и так же выполнил R2 метрику , и я вывел значимость каждого атрибута в моем датафрейме , самым значимым признаком оказался  Channel .


Выполнил прогнозирование для контрольной выборки с

помощью полученной модели. Определил качество модели на контрольной

выборке с помощью коэффициента детерминации (R2).

**Разработка API**


я считаю что у меня не получилось ее сделать , так как было не достаточно времени , и у меня не получилось сделать базу данных 